In [1]:
!python -V
#mlflow ui --backend-store-uri sqlite:///mlflow.db

Python 3.9.12


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Solar_Experiments")

2023/04/27 23:35:15 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/27 23:35:15 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location=('/home/akash/Documents/programming/luminous hackathon ppt/Luminous '
 'MLOps/02-experiment-tracking/mlruns/1'), creation_time=1682618715699, experiment_id='1', last_update_time=1682618715699, lifecycle_stage='active', name='Solar_Experiments', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

  
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    
    return df

In [8]:
df_train = read_dataframe('/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/01-intro/Solar Power Plant Data.csv')
df_val = read_dataframe('/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/01-intro/Solar Power Plant Data.csv')

In [9]:
len(df_train), len(df_val)

(8760, 8760)

In [10]:
numerical = ['WindSpeed', 'Sunshine', 'AirPressure', 'Radiation', 'AirTemperature', 'RelativeAirHumidity']
dv = StandardScaler()

train_dicts = df_train[numerical]
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[numerical]
X_val = dv.transform(val_dicts)

In [11]:
target = 'SystemProduction'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

897.1761873000875

In [13]:
with open('/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/02-experiment-tracking/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [14]:
with mlflow.start_run():

    mlflow.set_tag("developer", "akash")

    mlflow.log_param("train-data-path", "/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/01-intro/Solar Power Plant Data.csv")
    mlflow.log_param("valid-data-path", "/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/01-intro/Solar Power Plant Data.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/02-experiment-tracking/models/lin_reg.bin", artifact_path="models_pickle")

In [15]:
import xgboost as xgb

In [16]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [19]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[23:40:13] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:1188.39057                        
[1]	validation-rmse:864.99077                         
[2]	validation-rmse:632.73051                         
[3]	validation-rmse:465.58602                         
[4]	validation-rmse:344.03178                         
[5]	validation-rmse:254.78295                         
[6]	validation-rmse:189.93389                         
[7]	validation-rmse:143.33860                         
[8]	validation-rmse:107.89486                         
[9]	validation-rmse:81.44286                          
[10]	validation-rmse:62.11444                         
[11]	validation-rmse:48.19231                         
[12]	validation-rmse:38.05501                         
[13]	validation-rmse:30.16233                         
[14]	validation-rmse:24.40259                         
[15]	validation-rmse:20.06557                         
[

In [20]:
mlflow.xgboost.autolog(disable=True)

In [22]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.9563961406128333,
        'max_depth': 52,
        'min_child_weight': 0.6712731624297804,
        'objective': 'reg:linear',
        'reg_alpha': 0.008886705581960764,
        'reg_lambda': 0.0029066722892629564,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/02-experiment-tracking/models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/02-experiment-tracking/models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[23:49:02] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:77.71125
[1]	validation-rmse:3.70458
[2]	validation-rmse:0.18278
[3]	validation-rmse:0.01544
[4]	validation-rmse:0.00460
[5]	validation-rmse:0.00256
[6]	validation-rmse:0.00185
[7]	validation-rmse:0.00155
[8]	validation-rmse:0.00137
[9]	validation-rmse:0.00126
[10]	validation-rmse:0.00118
[11]	validation-rmse:0.00114
[12]	validation-rmse:0.00108
[13]	validation-rmse:0.00105
[14]	validation-rmse:0.00104
[15]	validation-rmse:0.00102
[16]	validation-rmse:0.00102
[17]	validation-rmse:0.00102
[18]	validation-rmse:0.00102
[19]	validation-rmse:0.00102
[20]	validation-rmse:0.00102
[21]	validation-rmse:0.00102
[22]	validation-rmse:0.00102
[23]	validation-rmse:0.00102
[24]	validation-rmse:0.00102
[25]	validation-rmse:0.00102
[26]	validation-rmse:0.00102
[27]	validation-rmse:0.00102
[28]	validation-rmse:0.00102
[29]	validation-rmse:0.00102
[30]	validation-rmse

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/01-intro/Solar Power Plant Data.csv")
        mlflow.log_param("valid-data-path", "/home/akash/Documents/programming/luminous hackathon ppt/Luminous MLOps/01-intro/Solar Power Plant Data.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2023/04/27 23:49:30 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
